In [11]:
#zuschneiden anhand der Skalenkoordinaten, um die Wäscheklammern abzuschneiden
import glob
from PIL import ImageDraw
from PIL import Image
import numpy as np
import re
import codecs
import math
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import ntpath

## Hyperparameter

In [12]:
# Pfade zu den Eingabedateien festlegen
input_data = {
    'dorsal': {
        'infolder': "DIP_images_fresh/images dorsal_2018/",
        'scale_data': "DIP_fresh_data/dorsal_2018/#Morph_DIP_dorsal_scale_20181124.TPS",
        'landmark': "DIP_fresh_data/dorsal_2018/#Morph_DIP_dorsal_view_20181125.TPS" 
    },
    'ventral': {
        'infolder': "DIP_images_fresh/images ventral_2018/",
        'scale_data': "DIP_fresh_data/ventral_2018/#Morph_DIP_ventral_scale_20181124.TPS",
        'landmark': "DIP_fresh_data/ventral_2018/#Morph_DIP_ventral_view_20181203.TPS" 
    }
}



In [13]:
# gewaehlter schluessel im oberen dict.
input_type = 'ventral'

infolder = input_data[input_type]['infolder']
inpaths = glob.glob(infolder + '*.jpg')
outfolder = f"out_all/landmarks/smallbox2/"
scale_data_path = input_data[input_type]['scale_data']


#Landmarkenkoordinaten ermitteln und anpassen an die Drehung
landmark_path = input_data[input_type]['landmark']

In [14]:
scale_data = {}

with codecs.open(landmark_path, 'r', encoding='utf-8',
                 errors='ignore') as f:
    
    landmarks = []
    
    for line in f:
        
        if line.startswith("IMAGE"):
            filename = line[len("IMAGE="):].rstrip()
            
            lm = np.array(landmarks)
            lm[:,1] = Image.open(infolder + filename).height - lm[:,1] #height - y-Koordinaten
            scale_data[filename] = {
                'landmarks': np.array(lm)
            }
            landmarks = []
        else:
            res = re.findall("\d+\.\d+", line)
        
            if len(res):
                lm = np.array(res).astype(np.float)
                landmarks.append( lm )
            
f.close() 


with codecs.open(scale_data_path, 'r', encoding='utf-8',
                 errors='ignore') as f:
    for line in f:
        line = line.rstrip() #\n entfernen
        if line.startswith("LM=2"): #bei jedem LM=2 folgen neue Informationen
            left = np.array(next(f).rstrip().split(' ')).astype(np.float).astype(np.int)
            right = np.array(next(f).rstrip().split(' ')).astype(np.float).astype(np.int)
            filename = next(f).replace('IMAGE=','').rstrip()
            
            scale_data[filename]['left'] = left
            scale_data[filename]['right']= right
f.close()

In [15]:
# gewaehlter schluessel im oberen dict.
input_type = 'dorsal'

infolder = input_data[input_type]['infolder']
inpaths = glob.glob(infolder + '*.jpg')
outfolder = f"out_all/landmarks/smallbox2/"
scale_data_path = input_data[input_type]['scale_data']


#Landmarkenkoordinaten ermitteln und anpassen an die Drehung
landmark_path = input_data[input_type]['landmark']


In [16]:
#scale_data = {}

with codecs.open(landmark_path, 'r', encoding='utf-8',
                 errors='ignore') as f:
    
    landmarks = []
    
    for line in f:
        
        if line.startswith("IMAGE"):
            filename = line[len("IMAGE="):].rstrip()
            
            lm = np.array(landmarks)
            lm[:,1] = Image.open(infolder + filename).height - lm[:,1] #height - y-Koordinaten
            scale_data[filename] = {
                'landmarks': np.array(lm)
            }
            landmarks = []
        else:
            res = re.findall("\d+\.\d+", line)
        
            if len(res):
                lm = np.array(res).astype(np.float)
                landmarks.append( lm )
            
f.close() 


with codecs.open(scale_data_path, 'r', encoding='utf-8',
                 errors='ignore') as f:
    for line in f:
        line = line.rstrip() #\n entfernen
        if line.startswith("LM=2"): #bei jedem LM=2 folgen neue Informationen
            left = np.array(next(f).rstrip().split(' ')).astype(np.float).astype(np.int)
            right = np.array(next(f).rstrip().split(' ')).astype(np.float).astype(np.int)
            filename = next(f).replace('IMAGE=','').rstrip()
            
            scale_data[filename]['left'] = left
            scale_data[filename]['right']= right
f.close()

In [19]:
np.save(f'all_scale_data.npy',scale_data)